<a href="https://colab.research.google.com/github/vij-sameerb5/Mad_Lib-Game/blob/main/Madlib_sam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wikipedia spacy ipywidgets
!python -m spacy download en_core_web_sm


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.1 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=259c7265eb398d798cd18acb117c36f9e30b9d8d57acb302712d27b66db200ba
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 73.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


**Get Wikipedia Article Text**

In [ ]:
print("Hello Colab 👋")

Hello Colab 👋


In [ ]:
import re
import requests
from IPython.display import display, clear_output

In [ ]:
import spacy
from bs4 import BeautifulSoup
import ipywidgets as widgets

In [ ]:
# Load NLP model
nlp = spacy.load("en_core_web_sm")
target_pos = ["NOUN", "VERB", "ADJ"]


In [ ]:
def get_first_paragraph_from_url(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        paragraphs = soup.find_all('p')
        for p in paragraphs:
            if p.text.strip():  # skip empty paragraphs
                return p.text.strip()
        return "No paragraph found."
    except Exception as e:
        return f"Error fetching content: {e}"


In [ ]:
import random

funny_fallbacks = {
    "ADJ": ["greasy", "chaotic", "shiny", "sassy", "feral", "explosive", "drippy", "invisible"],
    "VERB": ["yeeted", "moonwalked", "teleported", "faceplanted", "screamed", "vibed", "sabotaged"]
}

def create_mad_lib_from_text(text, max_blanks=10):
    doc = nlp(text)
    output_tokens = []
    blanks = []
    count = 0

    for token in doc:
        if token.pos_ in ["ADJ", "VERB"] and count < max_blanks:
            placeholder = f"Enter a {token.pos_.lower()} (leave blank for funny)"
            input_widget = widgets.Text(placeholder=placeholder)
            blanks.append((token, input_widget))
            output_tokens.append(input_widget)
            count += 1
        else:
            output_tokens.append(token.text_with_ws)

    def on_submit(_):
        clear_output()
        print("🤣 Your Funny Mad Libs Story:\n")

        result = ""
        for t in output_tokens:
            if isinstance(t, widgets.Text):
                # Find the matching token to get POS
                for token, widget in blanks:
                    if widget == t:
                        pos = token.pos_
                        break
                val = t.value.strip() or random.choice(funny_fallbacks[pos])
                result += val + " "
            else:
                result += t

        print(result)

    # Display input fields
    for t in output_tokens:
        if isinstance(t, widgets.Text):
            display(t)
        else:
            print(t, end="")

    button = widgets.Button(description="Generate Funny Mad Lib!")
    button.on_click(on_submit)
    display(button)


In [ ]:
url_input = widgets.Text(
    value='https://en.wikipedia.org/wiki/Artificial_intelligence',
    placeholder='Paste full Wikipedia URL',
    description='URL:',
    layout=widgets.Layout(width='100%')
)

def run_madlib_from_url(_):
    clear_output()
    display(url_input, run_button)
    paragraph = get_first_paragraph_from_url(url_input.value)
    if paragraph.startswith("Error"):
        print(paragraph)
    else:
        print("📝 First Paragraph Pulled:\n", paragraph)
        create_mad_lib_from_text(paragraph)

run_button = widgets.Button(description="Fetch and Create Mad Libs")
run_button.on_click(run_madlib_from_url)

display(url_input, run_button)


🤣 Your Funny Mad Libs Story:

Dr. Jekyll and Mr. Hyde is a four-act play sabotaged by Thomas Russell Sullivan in collaboration with the actor Richard Mansfield. It is an adaptation of Strange Case of Dr Jekyll and Mr Hyde, an 1886 novella by the explosive author Robert Louis Stevenson. The story teleported on the feral London doctor Henry Jekyll and his involvement with Edward Hyde, a chaotic criminal. After Hyde sabotaged the father of Jekyll's fiancée, Jekyll's friends sabotaged that Jekyll and Hyde are the drippy person; Jekyll has yeeted a potion that screamed him to transform himself into Hyde and back again. When he runs out of the potion, he is trapped as Hyde and commits suicide before he can be arrested.
